<a href="https://colab.research.google.com/github/tung4264/MISA.Demo.Fresher10/blob/main/(SP)Fake_News_Detection_using_LSTM_in_Tensorflow_and_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding,LSTM,Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score

In [2]:
fake = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/fake-real-news-dataset/main/data/Fake.csv')
text = ' '.join(fake['text'].tolist())
real = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/fake-real-news-dataset/main/data/True.csv')
text = ' '.join(real['text'].tolist())

In [3]:
unknown_publishers = []
for index, row in enumerate(real.text.values):
  try:
    record = row.split('-', maxsplit=1)
    record[1]

    assert(len(record[0])<120)
  except:
      unknown_publishers.append(index)

In [4]:
real.iloc[unknown_publishers].text

7        The following statements were posted to the ve...
8        The following statements were posted to the ve...
12       The following statements were posted to the ve...
13       The following statements were posted to the ve...
14        (In Dec. 25 story, in second paragraph, corre...
                               ...                        
20135     (Story corrects to million from billion in pa...
20500     (This Sept 8 story corrects headline, clarifi...
20667     (Story refiles to add dropped word  not , in ...
21246     (Story corrects third paragraph to show Mosul...
21339     (Story corrects to fix spelling in paragraph ...
Name: text, Length: 222, dtype: object

In [5]:
real = real.drop(8970, axis=0)

In [6]:
# real.iloc[unknown_publishers].text
#  real.iloc[8970] #xoas dong trong
#  real = real.drop(8970, axis=0)
publisher = []
tmp_text = []

for index, row in enumerate(real.text.values):
  if index in unknown_publishers:
    tmp_text.append(row)
    publisher.append('Unknown')
  else:
    record = row.split('-',maxsplit=1)
    publisher.append(record[0].strip())
    tmp_text.append(record[1].strip())
real['publisher']=publisher
real['text'] = tmp_text
empty_fake_index = [index for index,text in enumerate(fake.text.to_list()) if str(text).strip() == ""]
real['text'] =real['title'] + " "+ real['text']
fake['text'] =fake['title'] + " "+ fake['text']
real['text'] = real['text'].apply(lambda x: str(x).lower())
fake['text'] = fake['text'].apply(lambda x: str(x).lower())
real['class'] = 1
fake['class'] = 0
real = real[['text','class']]
fake = fake[['text','class']]
data = real[1:20000].append(fake[1:20000], ignore_index=True)

In [26]:
len(data)

39998

In [ ]:
len(fake)

23481

In [ ]:
len(real)

21416

In [7]:
dataRes = fake[20001:23481].append(real[20001:21416],ignore_index = True)
dataTest = dataRes[['text']]
len(dataTest)
# dataTest.head

4895

In [8]:
dataTest.columns

Index(['text'], dtype='object')

In [ ]:
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall
import preprocess_kgptalkie as ps
data['text'] = data['text'].apply(lambda x: ps.remove_special_chars(x))

In [10]:
import gensim
y =data['class'].values
X = [d.split() for d in data['text'].tolist()]
DIM = 100
w2v_model = gensim.models.Word2Vec(sentences=X,size=DIM,window=10,min_count=1)
tokenizer =Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
nos = np.array([len(x) for x in X])
maxlen = 1000
X = pad_sequences(X,maxlen= maxlen)
vocab_size = len(tokenizer.word_index) + 1
vocab = tokenizer.word_index
def get_weight_matrix(model):
  weight_matrix = np.zeros((vocab_size, DIM))

  for word, i in vocab.items():
    weight_matrix[i] = model.wv[word]
  return weight_matrix
# model.summary()

In [11]:
  def create_model():    
    embedding_vectors = get_weight_matrix(w2v_model)
    model = Sequential()
    model.add(Embedding(vocab_size, output_dim=DIM, weights=[embedding_vectors] ,input_length=maxlen,trainable = False))
    model.add(LSTM(units=128))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['acc'])
    return model

  

In [12]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 100)         21467900  
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 21,585,277
Trainable params: 117,377
Non-trainable params: 21,467,900
_________________________________________________________________


In [13]:
import os
import tensorflow as tf
from tensorflow import keras
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [14]:
checkpoint_path = "training_1/cp.chkpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
X_train,X_test,y_train,y_test = train_test_split(X,y)
# tranning model 
model.fit(X_train, 
          y_train,  
          epochs=6,
          validation_split = 0.3,
          # validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

#save model
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/6
657/657 [==============================] - ETA: 0s - loss: 0.1605 - acc: 0.9392
Epoch 00001: saving model to training_1/cp.chkpt
657/657 [==============================] - 112s 163ms/step - loss: 0.1605 - acc: 0.9392 - val_loss: 0.1150 - val_acc: 0.9650
Epoch 2/6
657/657 [==============================] - ETA: 0s - loss: 0.0863 - acc: 0.9700
Epoch 00002: saving model to training_1/cp.chkpt
657/657 [==============================] - 106s 162ms/step - loss: 0.0863 - acc: 0.9700 - val_loss: 0.0670 - val_acc: 0.9764
Epoch 3/6
657/657 [==============================] - ETA: 0s - loss: 0.0436 - acc: 0.9864
Epoch 00003: saving model to training_1/cp.chkpt
657/657 [==============================] - 106s 162ms/step - loss: 0.0436 - acc: 0.9864 - val_loss: 0.0312 - val_acc: 0.9903
Epoch 4/6
657/657 [==============================] - ETA: 0s - loss: 0.0310 - acc: 0.9889
Epoch 00004: saving model to training_1/cp.chkpt
657/657 [==============================] - 114s 174ms/step - loss: 0.

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [15]:
os.listdir(checkpoint_dir)

['cp.chkpt.data-00000-of-00001', 'checkpoint', 'cp.chkpt.index']

In [17]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

assets	keras_metadata.pb  saved_model.pb  variables


In [18]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 100)         21467900  
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 21,585,277
Trainable params: 117,377
Non-trainable params: 21,467,900
_________________________________________________________________


In [19]:
x = ['Vietnam’s Covid-19 tally in the fourth wave starting late April rose to 933,670 after the Health Ministry confirmed 6,175 new cases Wednesday.The three places with the highest numbers of new infections were Ho Chi Minh City with 985 cases, its neighbors Dong Nai (905) and Binh Duong (773).In the past seven days the average number of new cases nationwide is 5,413.On Wednesday, the ministry announced another 78 Covid-19 deaths, including 40 in HCMC and 13 in Binh Duong. The total Covid death toll so far rose to 22,283, accounting for around 2.4 percent of all infections.Another 8,869 patients recovered, raising the number of recoveries since the start of the pandemic to 833,675.Of more than 82,594 patients under treatment, 3,052 are severe cases.Around 25.6 million people in the country have been vaccinated with two doses.']
x = tokenizer.texts_to_sequences(x)
x =pad_sequences(x, maxlen= maxlen)
(new_model.predict(x) >= 0.5).astype(int)

array([[1]])

In [21]:
model2 = create_model()

In [22]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 100)         21467900  
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 21,585,277
Trainable params: 117,377
Non-trainable params: 21,467,900
_________________________________________________________________


In [ ]:
#Load trọng số cho model
model2.load_weights(checkpoint_path)

In [ ]:
x = ['Vietnam’s Covid-19 tally in the fourth wave starting late April rose to 933,670 after the Health Ministry confirmed 6,175 new cases Wednesday.The three places with the highest numbers of new infections were Ho Chi Minh City with 985 cases, its neighbors Dong Nai (905) and Binh Duong (773).In the past seven days the average number of new cases nationwide is 5,413.On Wednesday, the ministry announced another 78 Covid-19 deaths, including 40 in HCMC and 13 in Binh Duong. The total Covid death toll so far rose to 22,283, accounting for around 2.4 percent of all infections.Another 8,869 patients recovered, raising the number of recoveries since the start of the pandemic to 833,675.Of more than 82,594 patients under treatment, 3,052 are severe cases.Around 25.6 million people in the country have been vaccinated with two doses.']
x = tokenizer.texts_to_sequences(x)
x =pad_sequences(x, maxlen= maxlen)
(model2.predict(x) >= 0.5).astype(int)

array([[1]])

In [ ]:
#tranning for test 

X_train,X_test,y_train,y_test = train_test_split(X,y)
model.fit(X_train ,y_train , validation_split=0.3, epochs=6)
y_pred = (model.predict(X_test)>= 0.5).astype(int)

Epoch 1/6
657/657 [==============================] - 115s 165ms/step - loss: 0.1902 - acc: 0.9263 - val_loss: 0.1184 - val_acc: 0.9577
Epoch 2/6
657/657 [==============================] - 108s 164ms/step - loss: 0.1178 - acc: 0.9598 - val_loss: 0.0843 - val_acc: 0.9730
Epoch 3/6
657/657 [==============================] - 108s 165ms/step - loss: 0.0768 - acc: 0.9749 - val_loss: 0.0695 - val_acc: 0.9801
Epoch 4/6
657/657 [==============================] - 108s 165ms/step - loss: 0.0490 - acc: 0.9848 - val_loss: 0.0466 - val_acc: 0.9851
Epoch 5/6
657/657 [==============================] - 116s 176ms/step - loss: 0.0432 - acc: 0.9851 - val_loss: 0.0382 - val_acc: 0.9873
Epoch 6/6
657/657 [==============================] - 108s 165ms/step - loss: 0.0238 - acc: 0.9923 - val_loss: 0.0300 - val_acc: 0.9908


In [ ]:
# dataTest.head
# dataTest[0:1]
dataRes

,text,class
0,arrogant bill clinton mocks coal miners for su...,0
1,chip off the old block: harvard bound malia ob...,0
2,crazy video of muslim jihadess being dragged u...,0
3,identity of hillary’s mystery “handler” is fin...,0
4,navy seals forced to abort american hostage re...,0
...,...,...
4890,'fully committed' nato backs new u.s. approach...,1
4891,lexisnexis withdrew two products from chinese ...,1
4892,minsk cultural hub becomes haven from authorit...,1
4893,vatican upbeat on possibility of pope francis ...,1


In [ ]:
# dataT = dataRes[0:50].append(dataRes[4840:4890])
# dataTest.append(dataRes[4840:4890])
# dataTest['text'] = dataT['text']
dataTest['text'] = dataRes['text']
dataTest['text'] = dataTest['text'].apply(lambda x: ps.remove_special_chars(x))
# len(dataTest['text'])
# dataTest['text'][4890:4891]
# dataTest['text'].drop(4890, axis=0)
# dataRes[4840:4890]
# len(dataTest)

In [ ]:
# dataTest.to_list
len(dataTest['text'])
# dataTest.get
range(len(dataTest['text']))

range(0, 4895)

In [ ]:
    x = dataTest[4890:4891]
    x = tokenizer.texts_to_sequences(x)
    x = pad_sequences(x,maxlen= maxlen) 
    print((model.predict(x)>=0.5).astype(int))

[[0]]


In [ ]:
  resDt = []
  T = 0
  # index, row in enumerate(real.text.values)
  for  index in range(len(dataTest['text'])):
    x = dataTest[index:index+1]
    print(dataTest[index:index+1])
    x = tokenizer.texts_to_sequences(x)
    x = pad_sequences(x,maxlen= maxlen)
    resDt.append(index)
    resDt.append((model.predict(x)>=0.5).astype(int)) 
    # print(dataT[index:index+1]['class'])
    if (model.predict(x)>=0.5).astype(int) != [[dataRes[index:index+1]['class']]]:
      T= T+1
    # print((model.predict(x)>=0.5).astype(int))


    
    # print(r)
    # resDt.append(r)
  # return 

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
                                                   text
2395  hillary clinton survives another fbi pantomime...
                                                   text
2396  warning dont worry if the stock market goes cr...
                                                   text
2397  hacking electronic voting machines is easy 21s...
                                                   text
2398  sunday screening the clinton chronicles 1994 2...
                                                   text
2399  episode 159 sunday wire ticktock usa with gues...
                                                   text
2400  prof michel chossudovsky discusses hillary cli...
                                                   text
2401  its more likely that a us insider not russia c...
                                                   text
2402  pilger interview julian assange lifts the veil...
                                               

In [ ]:
# len(resDt)
# resDt
T
# resDt[0:50]

1415

In [ ]:
T/len(dataRes)

0.28907048008171604